In [1]:
# Install necessary packages
!pip install pandas numpy scikit-learn torch torchvision pillow


In [2]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image
from torchvision import transforms
import torch
from torch.utils.data import Dataset, DataLoader

# Define file paths
csv_file_path = r'C:\Users\Ayesha\Documents\Internship Projects\ITSOLERA\project\styles.csv'
image_folder = r'C:\Users\Ayesha\Documents\Internship Projects\ITSOLERA\project\images'

# Check if file exists and load data
if os.path.exists(csv_file_path):
    try:
        styles_df = pd.read_csv(
            csv_file_path,
            delimiter=',',
            encoding='utf-8',
            on_bad_lines='skip',
            quotechar='"',
            escapechar='\\'
        )
        print("CSV file loaded successfully.")
    except pd.errors.ParserError as e:
        print(f"ParserError: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
else:
    print("The file does not exist.")

# Display the first few rows and summary
print(styles_df.head())
print(styles_df.columns)
print(styles_df.info())

# Check for missing values
print(styles_df.isnull().sum())

# Drop or fill missing values
styles_df = styles_df.dropna(subset=['masterCategory', 'subCategory'])

# Encode categorical columns
label_encoder = LabelEncoder()
styles_df['masterCategory_encoded'] = label_encoder.fit_transform(styles_df['masterCategory'])

CSV file loaded successfully.
      id gender masterCategory subCategory  articleType baseColour  season  \
0  15970    Men        Apparel     Topwear       Shirts  Navy Blue    Fall   
1  39386    Men        Apparel  Bottomwear        Jeans       Blue  Summer   
2  59263  Women    Accessories     Watches      Watches     Silver  Winter   
3  21379    Men        Apparel  Bottomwear  Track Pants      Black    Fall   
4  53759    Men        Apparel     Topwear      Tshirts       Grey  Summer   

     year   usage                             productDisplayName  
0  2011.0  Casual               Turtle Check Men Navy Blue Shirt  
1  2012.0  Casual             Peter England Men Party Blue Jeans  
2  2016.0  Casual                       Titan Women Silver Watch  
3  2011.0  Casual  Manchester United Men Solid Black Track Pants  
4  2012.0  Casual                          Puma Men Grey T-shirt  
Index(['id', 'gender', 'masterCategory', 'subCategory', 'articleType',
       'baseColour', 'season

In [3]:
from PIL import Image
from torchvision import transforms

# Define image preprocessing function
def preprocess_image(image_path, target_size=(128, 128)):
    """Load an image file and preprocess it."""
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(target_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return preprocess(image)

# Create a custom Dataset class
class ClothingDataset(Dataset):
    def __init__(self, dataframe, image_folder, transform=None):
        self.dataframe = dataframe
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_path = os.path.join(self.image_folder, f'{row["id"]}.jpg')
        image = preprocess_image(image_path) if os.path.exists(image_path) else torch.zeros(3, 128, 128)
        label = row["masterCategory_encoded"]
        return image, label

# Split data into training and validation sets
train_df, val_df = train_test_split(styles_df, test_size=0.2, random_state=42)

# Create datasets and dataloaders
train_dataset = ClothingDataset(train_df, image_folder)
val_dataset = ClothingDataset(val_df, image_folder)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [4]:
import torch.nn as nn
import torch.optim as optim

# Define the CNN model
class ClothingClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ClothingClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.model(x)

num_classes = len(styles_df['masterCategory_encoded'].unique())
model = ClothingClassifier(num_classes=num_classes)


In [6]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).long()  # Convert labels to torch.long
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
    
    # Validation
    model.eval()
    corrects = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device).long()  # Convert labels to torch.long
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            corrects += torch.sum(preds == labels.data).item()
            total += labels.size(0)
    
    val_accuracy = corrects / total
    print(f'Validation Accuracy: {val_accuracy:.4f}')
    
# Save the model
torch.save(model.state_dict(), 'fashion_outfit_recommendation_model.pth')


Epoch 1/10, Loss: 0.1777
Validation Accuracy: 0.9778
Epoch 2/10, Loss: 0.0899
Validation Accuracy: 0.9788
Epoch 3/10, Loss: 0.0601
Validation Accuracy: 0.9835
Epoch 4/10, Loss: 0.0498
Validation Accuracy: 0.9857
Epoch 5/10, Loss: 0.0376
Validation Accuracy: 0.9836
Epoch 6/10, Loss: 0.0264
Validation Accuracy: 0.9849
Epoch 7/10, Loss: 0.0260
Validation Accuracy: 0.9865
Epoch 8/10, Loss: 0.0208
Validation Accuracy: 0.9847
Epoch 9/10, Loss: 0.0196
Validation Accuracy: 0.9795
Epoch 10/10, Loss: 0.0245
Validation Accuracy: 0.9849


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the CNN model architecture again
class CNNClassifier(nn.Module):
    def __init__(self, num_classes):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [12]:
# Load the state dict
state_dict = torch.load('fashion_outfit_recommendation_model.pth')

# Remove 'model.' prefix from the keys
new_state_dict = {}
for k, v in state_dict.items():
    name = k.replace('model.', '')  # Remove 'model.' prefix
    new_state_dict[name] = v

# Load the modified state dict
model.load_state_dict(new_state_dict)

# Set model to evaluation mode
model.to(device)
model.eval()


C:\Users\Ayesha\AppData\Local\Temp\ipykernel_12492\3321150632.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('fashion_outfit_recommendation_mode

Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU()
  (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (9): Flatten(start_dim=1, end_dim=-1)
  (10): Linear(in_features=32768, out_features=512, bias=True)
  (11): ReLU()
  (12): Dropout(p=0.5, inplace=False)
  (13): Linear(in_features=512, out_features=7, bias=True)
)

In [14]:
import torch
import torch.nn as nn

# Correct model architecture
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    
    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    
    nn.Flatten(),
    
    # Adjust the input size here
    nn.Linear(128 * 16 * 16, 512),  # 128 * 16 * 16 = 32768
    nn.ReLU(),
    nn.Dropout(0.5),
    
    nn.Linear(512, len(styles_df['masterCategory_encoded'].unique()))  # Output layer
)

# Load the saved state dict
state_dict = torch.load('fashion_outfit_recommendation_model.pth')

# Remove 'model.' prefix from the keys in the state dictionary
new_state_dict = {k.replace('model.', ''): v for k, v in state_dict.items()}

# Load the updated state dictionary into the model
model.load_state_dict(new_state_dict)

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set the model to evaluation mode
model.eval()

print("Model loaded successfully.")


Model loaded successfully.


C:\Users\Ayesha\AppData\Local\Temp\ipykernel_12492\951507578.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('fashion_outfit_recommendation_mode

In [15]:
# Load the saved state dict with weights_only=True
state_dict = torch.load('fashion_outfit_recommendation_model.pth', weights_only=True)

# Remove 'model.' prefix from the keys in the state dictionary
new_state_dict = {k.replace('model.', ''): v for k, v in state_dict.items()}

# Load the updated state dictionary into the model
model.load_state_dict(new_state_dict)

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set the model to evaluation mode
model.eval()

print("Model loaded successfully without warnings.")


Model loaded successfully without warnings.


In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np


In [29]:
class OutfitDataset(Dataset):
    def __init__(self, image_data, metadata, labels, transform=None):
        self.image_data = image_data  # List of images (file paths or tensors)
        self.metadata = metadata      # Tabular data for each image
        self.labels = labels          # Outfit compatibility labels (1 if compatible, 0 otherwise)
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.image_data[idx]
        if self.transform:
            image = self.transform(image)
        
        metadata = self.metadata[idx]
        label = self.labels[idx]

        return image, metadata, label


In [30]:
class ImageFeatureExtractor(nn.Module):
    def __init__(self):
        super(ImageFeatureExtractor, self).__init__()
        # Use a pre-trained ResNet model
        self.resnet = models.resnet50(pretrained=True)
        # Remove the final layer
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])

    def forward(self, x):
        x = self.resnet(x)
        x = x.view(x.size(0), -1)  # Flatten the output
        return x


In [31]:
class MetadataProcessor(nn.Module):
    def __init__(self, input_size, hidden_size=128):
        super(MetadataProcessor, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x


In [32]:
class OutfitRecommendationModel(nn.Module):
    def __init__(self, metadata_input_size, hidden_size=128):
        super(OutfitRecommendationModel, self).__init__()
        # Image feature extractor
        self.image_extractor = ImageFeatureExtractor()

        # Metadata processor
        self.metadata_processor = MetadataProcessor(metadata_input_size, hidden_size)

        # Final layers combining both image and metadata features
        combined_input_size = 2048 + hidden_size  # ResNet outputs 2048-dim vector
        self.fc1 = nn.Linear(combined_input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)  # Output compatibility score (1 for compatible, 0 for not)

    def forward(self, image, metadata):
        # Process image features
        image_features = self.image_extractor(image)

        # Process metadata features
        metadata_features = self.metadata_processor(metadata)

        # Concatenate image and metadata features
        combined_features = torch.cat((image_features, metadata_features), dim=1)

        # Final prediction layers
        x = F.relu(self.fc1(combined_features))
        x = torch.sigmoid(self.fc2(x))  # Output a score between 0 and 1
        return x


In [36]:
def train_model(model, dataloader, num_epochs=10, learning_rate=0.001):
    criterion = nn.BCELoss()  # Binary Cross Entropy Loss for compatibility
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, metadata, labels in dataloader:
            # Forward pass
            outputs = model(images, metadata)
            loss = criterion(outputs.squeeze(), labels.float())

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader)}')

    print('Training completed!')


In [37]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

def preprocess_metadata(metadata_df):
    # Assuming metadata_df is a pandas DataFrame containing the columns articleType, baseColour, etc.
    label_encoders = {}
    encoded_metadata = np.zeros_like(metadata_df)

    for col in metadata_df.columns:
        le = LabelEncoder()
        encoded_metadata[col] = le.fit_transform(metadata_df[col])
        label_encoders[col] = le

    return encoded_metadata, label_encoders


In [38]:
# Example dummy data
image_data = torch.randn(100, 3, 224, 224)  # 100 random image tensors
metadata = torch.randn(100, 5)  # 5 tabular features (articleType, baseColour, etc.)
labels = torch.randint(0, 2, (100,))  # Random compatibility labels (0 or 1)

# Create dataset and dataloader
dataset = OutfitDataset(image_data, metadata, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize and train the model
metadata_input_size = metadata.shape[1]
model = OutfitRecommendationModel(metadata_input_size)
train_model(model, dataloader, num_epochs=5)


C:\Users\Ayesha\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Ayesha\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Ayesha/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [01:44<00:00, 981kB/s] 


Epoch [1/5], Loss: 0.8879333883523941
Epoch [2/5], Loss: 0.48354943096637726
Epoch [3/5], Loss: 0.17327819392085075
Epoch [4/5], Loss: 0.2069761073216796
Epoch [5/5], Loss: 0.7092091618105769
Training completed!


In [39]:
torch.save(model.state_dict(), 'fashion_outfit_recommendation_model.pth')
